In [1]:
import plotly.graph_objects as go
from pymongo import MongoClient, DESCENDING, ASCENDING
import pandas as pd
import numpy as np
import ta
import logging

### Add New Time Interval

In [2]:
def fetch_data(symbol):
    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017/')
    db = client['historic_data']
    collection = db['daily_stock_price']
    # Get one symbol data from the collection
    df = pd.DataFrame(list(collection.find({'symbol':symbol},{'_id': 0}).sort('date', ASCENDING)))
    return df

In [3]:
new_interval_record = {}

def create_new_interval_data(df, interval):
    new_df = df.set_index('date')
    new_df = new_df.groupby(pd.Grouper(freq=interval)).agg({'open': 'first',
                                                'high': 'max', 
                                                'low': 'min', 
                                                'close': 'last', 
                                                'volume': 'sum'})
    new_df.reset_index(inplace=True)
    new_df = new_df.dropna()
    return new_df


In [4]:
# Add bull and bear features in the dataframe
class add_features:
    def __init__(self, df):
        self.df = df.copy()

    def add_candlestick(self):
        self.df["BodyDiff"] = abs(self.df["open"] - self.df["close"])
        self.df["CandleStickType"] = np.where(self.df["open"] < self.df["close"], "green", "red")
        return self.df

    def continuous_increase(self, windows=3):
        for i in range(1, windows + 1):
            self.df[f"close_t-{i}"] = self.df["close"].shift(i)

        self.df['Incremental_High'] = (self.df['close'] > self.df['close_t-1']) \
                                        & (self.df['close_t-1'] > self.df['close_t-2']) \
                                        & (self.df['close_t-2'] > self.df['close_t-3'])
                                        
        self.df = self.df.drop(columns=[f"close_t-{i}" for i in range(1, windows + 1)])
        return self.df

    def macd_golden_cross(self):
        self.df['MACD_GOLDEN_CROSS'] = (self.df['MACD'] > self.df['MACD_SIGNAL']) & (self.df['MACD'] < 0)
        return self.df
    
    # Add Technical Indicators
    def add_technical(self):
        
        # Add ema dual channels technical indicators
        self.df['8EMA'] = ta.trend.ema_indicator(self.df['close'], window=8)
        self.df['13EMA'] = ta.trend.ema_indicator(self.df['close'], window=13)
        self.df['144EMA'] = ta.trend.ema_indicator(self.df['close'], window=144)
        self.df['169EMA'] = ta.trend.ema_indicator(self.df['close'], window=169)

        # Add MACD technical indicator
        self.df['MACD'] = ta.trend.macd(self.df['close'], 
                                        window_slow=26, 
                                        window_fast=12)
        
        self.df['MACD_SIGNAL'] = ta.trend.macd_signal(self.df['close'], 
                                                    window_slow=26, 
                                                    window_fast=12, 
                                                    window_sign=9)
        
        self.df['MACD_HIST'] = ta.trend.macd_diff(self.df['close'], 
                                                window_slow=26, 
                                                window_fast=12, 
                                                window_sign=9)
        
        # Add ATR technical indicator
        self.df["atr"] = ta.volatility.AverageTrueRange(high=self.df.high, 
                                                        low=self.df.low, 
                                                        close=self.df.close).average_true_range()
        
        self.df["atr"] = self.df.atr.rolling(window=30).mean()
        
        return self.df
    
    def apply(self):
        self.add_technical()
        self.add_candlestick()
        self.continuous_increase()
        self.macd_golden_cross()
        self.add_ema_band()
        
        # Use last 1000 rows of the dataframe
        self.df = self.df.tail(3000) if len(self.df) > 3000 else self.df
        return self.df


In [5]:
class add_alert:
    
    def __init__(self, df):
        self.df = df.copy()
        self.df['MACD_Alert'] = -1  
        self.df['Engulf_Alert'] = -1 
        self.df['dual_channel_Alert'] = -1  
        self.df['382_Alert'] = -1  
        self.window = 3  # Number of days to compare the stock price
        
    def engulf_alert(self):
        
        # Previous Candle
        prev_open = self.df['open'].shift(1)
        prev_close = self.df['close'].shift(1)
        
        # Current Candle
        current_open = self.df['open']
        current_close = self.df['close']

        # Bullish Engulfing: current green candle engulfs previous red candle
        bullish_engulfing = (
            (prev_close < prev_open) &  # Previous candle was red
            (current_close > current_open) &  # Current candle is green
            (current_open < prev_close) & 
            (current_close > prev_open)
        )

        # Bearish Engulfing: current red candle engulfs previous green candle
        bearish_engulfing = (
            (prev_close > prev_open) &  # Previous candle was green
            (current_close < current_open) &  # Current candle is red
            (current_open > prev_close) & 
            (current_close < prev_open)
        )

        # Apply the conditions to the DataFrame
        self.df.loc[bullish_engulfing, 'Engulf_Alert'] = 1
        self.df.loc[bearish_engulfing, 'Engulf_Alert'] = 0
        
        return self.df

    def macd_alert(self):
        
        # Pre-compute conditions
        macd_above_signal = self.df['MACD'] > self.df['MACD_SIGNAL']
        macd_increasing = self.df['MACD'].diff() > 0
        macd_below_zero = self.df['MACD'] >= 0

        # Bullish condition
        bullish_macd = (
            macd_above_signal &
            macd_increasing&
            macd_below_zero
        )
        
        # Bearish condition
        bearish_macd = (
            (self.df['MACD'] < self.df['MACD_SIGNAL']) &
            (self.df['MACD'] > self.df['MACD'].shift(-1))  
        )

        # Apply the conditions to the DataFrame
        self.df.loc[bullish_macd, 'MACD_Alert'] = 1
        self.df.loc[bearish_macd, 'MACD_Alert'] = 0

        return self.df

    def dual_channel(self):
        
        # Confirm Technical Indicators Conditions
        ema_8_gt_ema_13 = self.df['8EMA'] > self.df['13EMA']
        ema_13_gt_ema_169 = self.df['13EMA'] > self.df['169EMA']
        ema_8_gt_ema_144 = self.df['8EMA'] > self.df['144EMA']
        
        # Confirm Close price action
        close_ge_ema_13 = self.df['close'] >= self.df['13EMA']
        close_ge_ema_144 = self.df['close'] >= self.df['144EMA']
        
        # Confirm Candle fully above 13 EMA
        low_ge_ema_13 = self.df['low'] >= self.df['13EMA'] 
        green_candle = self.df['CandleStickType'] == 'green'    
        
        # Confirm Candle in selected range
        open_in_slow_ema = self.df['open'].between(self.df['169EMA_Lower'], self.df['169EMA_Upper'])
        
        # Confirm Candle type
        green_candle = self.df['CandleStickType'] == 'green'
        
        # Bullish Conditions
        
        bullish_scenario_1 = (
            ema_8_gt_ema_13 & 
            ema_13_gt_ema_169 & 
            close_ge_ema_13 &
            low_ge_ema_13 &
            green_candle
        )
        
        # Bearish Condition
        bearish_scenario_1 = (
            (self.df['open'] < self.df['13EMA']) &
            (self.df['close'] < self.df['13EMA']) &
            (self.df['13EMA'] < self.df['8EMA'])
        )
        # Bullish Conditions 2
        bullish_scenario_2 = ( 
            ema_8_gt_ema_144 &
            ema_13_gt_ema_169 &
            close_ge_ema_144 &
            open_in_slow_ema &
            green_candle
            )
        
        # Bearish Condition 2
        bearish_scenario_2 = (
            (self.df['open'] < self.df['13EMA']) &
            (self.df['close'] < self.df['13EMA']) &
            (self.df['8EMA'] < self.df['13EMA']) &
            ~ open_in_slow_ema
        )

        # Apply conditions on different price scenarios
        
        self.df.loc[bullish_scenario_1 | bullish_scenario_2, 'dual_channel_Alert'] = 1
        self.df.loc[bearish_scenario_1 | bearish_scenario_2, 'dual_channel_Alert'] = 0
        
        return self.df

    def bullish_382_alert(self):
        # Calculate Fibonacci 38.2% retracement level
        diff = self.df['high'] - self.df['low']
        fib_382_level = self.df['high'] - 0.382 * diff
        open_above_fib_382 = self.df['open'] > fib_382_level
        
        # Apply condition
        self.df.loc[open_above_fib_382, '382_Alert'] = 1
        self.df.loc[~open_above_fib_382, '382_Alert'] = 0
        
        return self.df

    def apply(self):
        self.engulf_alert()
        self.macd_alert()
        self.dual_channel()
        self.bullish_382_alert()
        return self.df

In [6]:
def insert_technical_data(db, tech_collection_name, symbol, df, interval, current_date):
    # Check for last record in the collection
    last_record = db[tech_collection_name].find_one({"symbol": symbol, "interval": interval},
                                                    sort=[('timestamp', DESCENDING)])
    if last_record:
        last_date_in_db = pd.to_datetime(last_record['date']).strftime('%Y-%m-%d')
        if last_date_in_db == current_date:
            logging.info(f"Data for {symbol} is up to date")
            return
        new_records_df = df[df['date'] > last_date_in_db]
    else:
        new_records_df = df

    if not new_records_df.empty:
        new_records_df["symbol"] = symbol
        new_records_df["interval"] = interval
        new_records_df["timestamp"] = pd.to_datetime(new_records_df["date"])
        db[tech_collection_name].insert_many(new_records_df.to_dict(orient='records'))
        logging.info(f"Inserted {len(new_records_df)} records into {tech_collection_name} collection")


In [7]:
def process_made(db, tech_collection_name, new_intervals, current_date):
    distinct_symbols = db[tech_collection_name].distinct('symbol')
    for symbol in distinct_symbols:
        df = fetch_data(symbol)       
        for interval in new_intervals:
            logging.info(f"Adding new interval {interval} for symbol {symbol}...")
            
            new_df = create_new_interval_data(df, interval)
            
            logging.info(f"Processing completed for {symbol} in {interval} interval")
            
            # Check if the data is already in the collection
            last_record = db[tech_collection_name].find_one({"symbol": symbol, "interval": interval},
                                                            sort=[('timestamp', DESCENDING)])
            current_date_dt = pd.to_datetime(current_date)
            if last_record:
                # Ensure the date is in pandas datetime format
                last_date_in_db = pd.to_datetime(last_record['timestamp']).strftime('%Y-%m-%d')
                
                # Check if the data is up to date
                if last_date_in_db == current_date_dt:
                    logging.info(f"The made {interval} Data for {symbol} is up to date")
                    continue
                else:
                    # Take only the fetched record is newer
                    new_records_df = new_df[new_df['date'] > last_date_in_db]
            # If the symbol does not exist in the technical collection
            else:
                new_records_df = new_df
            # Insert the new interval data accordingly
            if not new_records_df.empty:
                insert_technical_data(db, tech_collection_name, symbol, new_records_df, interval,current_date_dt)
                logging.info(f"Inserted new interval data for {symbol} in {interval} interval")
                
process_made(MongoClient('localhost')['historic_data'], 'processed_stock_data', ['2D','3D','4D','5D'], '2024-10-11')

### Defien the Stock Price Movement Velocity

In [8]:
def fetch_and_prepare_data(symbol, interval):
    """
    Fetch and prepare data for a given stock symbol.
    
    Parameters:
    symbol (str): The stock symbol to fetch data for.
    
    Returns:
    pd.DataFrame: A DataFrame containing the prepared data.
    """
    # Fetch all interval data for the selected stock
    lst = list(MongoClient('localhost')['data_warehouse']['processed_data']\
        .find({'symbol': symbol, 'interval': interval},{'_id': 0, 'close': 1, 'open': 1, 'low': 1, 'high': 1, 'date': 1, "interval": 1, "13EMA": 1,"169EMA": 1, '169EMA_Lower':1, '169EMA_Upper':1})\
            .sort([('interval', DESCENDING), ('date', ASCENDING)]))
    
    # Convert the list to a dataframe
    df = pd.DataFrame(lst)
    
    # Drop weekly data
    df = df[df['interval'] != 'weekly']

    # Drop rows with missing values
    df = df.dropna()

    return df


#### Support Loss Function

Let:
- $C_i$ be the closing price at time step $i$
- ${EMA}_{13,i}$ be the 13-period Exponential Moving Average (EMA) at time step $i$
- $N$ be the total number of time steps in the interval

The **support loss** is calculated as the sum of penalties for all time steps where the closing price \( C_i \) is below the 13 EMA:

$$
\text{Support Loss} = \sum_{i=1}^{N} \max(0,  \text{EMA}_{13,i} - C_i)
$$

Where:
- If $C_i \geq \text{EMA}_{13, i}$, the loss for that time step is 0 (no penalty).
- If $C_i < \text{EMA}_{13, i}$, the loss for that time step is the absolute difference ${EMA}_{13,i} - C_i$.

In [9]:
class VelocityFinder:
    def __init__(self, data):
        self.data = data

    def support_loss(self, closing_prices, ema_13):
        """
        Custom loss function to find intervals where the 13 EMA acts as support.
        Penalizes any instance where the closing price falls below the 13 EMA.
        """
        differences = closing_prices - ema_13
        penalties = np.where(differences < 0, np.abs(differences), 0)
        return np.sum(penalties)

    def find_best_interval(self):
        """
        Find the interval with the lowest time-weighted 'price velocity', 
        i.e., the one where the closing price fits the 13 EMA line the best, for the last 60 days.
        """
        intervals = self.data['interval'].unique()

        min_loss = float('inf')
        best_interval = None

        # Loop through each interval to find the best fitting one
        for interval in intervals:
            interval_data = self.data[self.data['interval'] == interval].tail(60)
            if not interval_data.empty:
                closing_prices = interval_data['close'].values
                ema_13 = interval_data['13EMA'].values
                loss = self.support_loss(closing_prices, ema_13)

                if loss < min_loss:
                    min_loss = loss
                    best_interval = interval

        # Find the date range of 60 candles in the best interval
        interval_date_start = self.data[self.data['interval'] == best_interval]['date'].values[-60]
        interval_date_end = self.data[self.data['interval'] == best_interval]['date'].values[-1]

        # Attach the best interval as the best_velocity for the rows with dates between interval_date_start and interval_date_end
        self.data.loc[
            (self.data['date'] >= interval_date_start) & 
            (self.data['date'] <= interval_date_end),
            'best_velocity'
        ] = best_interval

    def run(self):
        self.find_best_interval()
        
        for interval in self.data['interval'].unique():
            self.data.loc[self.data['interval'] == interval] = self.data.loc[self.data['interval'] == interval].ffill()
        return self.data
        
data = fetch_and_prepare_data('SOXX', 1)
vf = VelocityFinder(data)
best_velocity = vf.run()

### EMA Support Testing Alert

- Add columns to detect whether the closing price enters the predefined 169 EMA band.
- Detect if the closing price is able to recover and move back above the 169 EMA after touching the band.

For example:
* 169 EMA Band Calculation
$$
{EMA_{169} Band} = EMA_{169} \times (1 \pm \epsilon)
$$
Where  $\epsilon$ is a small percentage to define the upper and lower bounds of the band.

* Recovery Indicator
Let ${Recovery}_{i}$ be an indicator where $i$ is either 0 or 1:
- $1$: The stock price recovered after touching the 169 EMA band and moved above the upper bound within the specified number of days.
- $0$: The stock price did not recover after touching the 169 EMA band.

In [18]:
def ema_support_alert(df, judgement_days=6, decision_days=3, epilson=0.05):
    ema_periods = [13, 169]  # Example EMA periods

    for ema_period in ema_periods:
        df[f'{ema_period}EMA_Lower'] = df[f'{ema_period}EMA'] * (1 - epilson)
        df[f'{ema_period}EMA_Upper'] = df[f'{ema_period}EMA'] * (1 + epilson)

        df[f'above_{ema_period}_emas'] = df['close'] > df['169EMA']
        df[f'below_{ema_period}_emas'] = df['close'] < df['169EMA']
        df[f'inside_{ema_period}_ema_band'] = (df['close'] > df[f'{ema_period}EMA_Lower']) & (df['close'] < df[f'{ema_period}EMA_Upper'])

        ema_column = f'{ema_period}EMA'
        df[f'{ema_period}EMA_Lower'] = df[ema_column] * (1 - epilson)
        df[f'{ema_period}EMA_Upper'] = df[ema_column] * (1 + epilson)

    ema13_alert_date = 0
    ema169_alert_date = 0
        
    for i in df.index:
        if i - judgement_days < df.index[0]:
            continue  # Skip the iteration if not enough history is available for comparison

        # Ensure you are operating on matching ranges for indices
        close_range = df.loc[i - decision_days: i, 'close']
        open_range = df.loc[i - decision_days: i, 'open']
        ema13_range = df.loc[i - decision_days: i, '13EMA']
        ema169_range = df.loc[i - decision_days: i, '169EMA']
        
        # Only proceed if the required ranges have identical lengths and indices
        if len(close_range) == len(ema13_range):
            # Check conditions for alerts
            if (df.loc[i - judgement_days: i - decision_days, 'inside_13_ema_band']).any() \
                and df.loc[i - judgement_days: i - decision_days, 'above_169_emas'].all() \
                and df.loc[i - judgement_days: i - decision_days, '13EMA_Upper'].any():
                
                # Condition for "13ema_recovery"
                if (close_range >= ema13_range).all() \
                    and (open_range >= ema13_range).all() \
                    and (df.loc[i, 'close'] >= df.loc[i - decision_days: i, 'close']).all() \
                    and (df.loc[i, 'open'] >= df.loc[i - decision_days: i, 'open']).all():
                    
                    if ema13_alert_date == 0 or not (df.loc[ema13_alert_date : i, 'velocity_alert'] == '13ema_recovery').any():
                        df.loc[i, 'velocity_alert'] = '13ema_recovery'
                        ema13_alert_date = i

                # Condition for "13ema_recover_failed"
                elif (close_range <= df.loc[i - decision_days : i, '13EMA']).all() \
                    and (open_range <= df.loc[i - decision_days : i, '13EMA']).all():
                    
                    if ema13_alert_date == 0 or not (df.loc[ema13_alert_date : i, 'velocity_alert'] == '13ema_recover_failed').any():
                        df.loc[i, 'velocity_alert'] = '13ema_recover_failed'
                        ema13_alert_date = i
        else:
            print(f"Mismatched lengths for index range at {i}")

        if len(close_range) == len(ema169_range):
            if (df.loc[i - judgement_days : i - decision_days, 'inside_169_ema_band']).any() \
                and df.loc[i - judgement_days : i - decision_days, 'above_169_emas'].any()\
                    and df.loc[i - judgement_days : i - decision_days, '169EMA_Upper'].any():

                if (close_range >= ema169_range).all() \
                    and (open_range >= ema169_range).all() \
                    and (df.loc[i, 'close'] >= df.loc[i - decision_days : i, 'close']).all() \
                    and (df.loc[i, 'open'] >= df.loc[i - decision_days : i, 'open']).all():
                    
                    if ema169_alert_date == 0 or not (df.loc[ema169_alert_date : i, 'velocity_alert'] == '169ema_recovery').any():
                        df.loc[i, 'velocity_alert'] = '169ema_recovery'
                        ema169_alert_date = i

                elif (close_range <= df.loc[i - decision_days : i, '169EMA']).all() \
                    and (open_range <= df.loc[i - decision_days : i, '169EMA']).all():
                    
                    if ema169_alert_date == 0 or not (df.loc[ema169_alert_date : i, 'velocity_alert'] == '169ema_recover_failed').any():
                        df.loc[i, 'velocity_alert'] = '169ema_recover_failed'
                        ema169_alert_date = i

    for ema_period in ema_periods:
        df = df.drop(columns=[f'{ema_period}EMA_Lower', f'{ema_period}EMA_Upper', f'above_{ema_period}_emas', f'below_{ema_period}_emas', f'inside_{ema_period}_ema_band'])
        
    return df

# Test case
nvda_df = fetch_and_prepare_data('TSLA', 1)
test_ema_support_alert_df = ema_support_alert(nvda_df)

# Get the rows with non-null alerts
non_null_alerts = test_ema_support_alert_df.loc[test_ema_support_alert_df['velocity_alert'].notnull()]


In [19]:
import plotly.graph_objects as go

# Plot the candlestick data
fig = go.Figure(data=[go.Candlestick(x=nvda_df['date'],
                                    open=nvda_df['open'],
                                    high=nvda_df['high'],
                                    low=nvda_df['low'],
                                    close=nvda_df['close'],
                                    name='Candlestick')])

# Add 13 EMA and 169 EMA
fig.add_scatter(x=test_ema_support_alert_df['date'], y=test_ema_support_alert_df['13EMA'], mode='lines', name='13 EMA')
fig.add_scatter(x=test_ema_support_alert_df['date'], y=test_ema_support_alert_df['169EMA'], mode='lines', name='169 EMA')

# Add annotations for alerts
for i, row in non_null_alerts.iterrows():
    fig.add_annotation(
        x=row['date'],
        y=row['close'],
        text=row['velocity_alert'],
        showarrow=True,
        arrowhead=1
    )

# Enable x-axis rangeslider for dynamic selection
fig.update_layout(
    xaxis=dict(
        title='Date',
        rangeslider=dict(visible=True),  # Enable rangeslider for zooming in/out
        type="date"
    ),
    yaxis=dict(
        title='Price',
        fixedrange=False  # Allow y-axis to auto-scale
    ),
    showlegend=True
)

# Dynamically adjust the y-axis based on visible x-axis range
def scale_y_axis_on_zoom(fig, data, x_min, x_max):
    # Filter the data within the selected x-axis range
    filtered_data = data[(data['date'] >= x_min) & (data['date'] <= x_max)]
    y_min = filtered_data['low'].min()
    y_max = filtered_data['high'].max()
    
    # Update y-axis range based on the filtered data
    fig.update_yaxes(range=[y_min, y_max])

# Set callback for relayout to dynamically scale the y-axis
fig.update_layout(
    xaxis_rangeslider_visible=True,
    autosize=True
)

# Add a callback that will automatically update the y-axis when zooming
fig.update_layout(
    dragmode='zoom',
    xaxis_rangeslider_visible=True
)

# Automatically rescale y-axis upon zoom or range update
fig.update_xaxes(rangeselector=dict(
    buttons=list([
        dict(count=1, label="1m", step="month", stepmode="backward"),
        dict(count=3, label="3m", step="month", stepmode="backward"),
        dict(count=6, label="6m", step="month", stepmode="backward"),
        dict(step="all")
    ])
))

fig.show()


In [170]:
test_ema_support_alert_df['velocity_alert'].value_counts()

velocity_alert
13ema_recovery           9
13ema_recover_failed     8
169ema_recover_failed    2
169ema_recovery          2
Name: count, dtype: int64

### Predefined Velocity Support Testing Alert

- Assume the stock price above the 13 EMA and 169 EMA conisdered as strong upward trend
- Assume the stock price touches the EMA from below or above is negotiating with the market and not decide to go upward or downward 
- All stock price possess moves at a interval that satisfies the above assumption the best, finding the best fitting interval (velocity) is the major task.


In [16]:
def velocity_alert(df, window=3):
    # Condition where closing price above the 13 EMA and 169 EMA at the same time
    df['above_13EMA'] = df['close'] > df['13EMA']
    df['above_169EMA'] = df['close'] > df['169EMA']

    # Condition where closing price below the 13 EMA and 169 EMA
    df['below_13EMA'] = df['close'] < df['13EMA']
    df['below_169EMA'] = df['close'] < df['169EMA']

    # Condition where closing price is sandwiched between the 13 EMA and 169 EMA
    df['between_13_169EMA'] = (df['close'] <= df['13EMA']) & (df['close'] >= df['169EMA'])

    # Loop through the rows to calculate the velocity alerts
    for i in df.index:
        # Skip rows before the window starts to avoid indexing errors
        if i - window < df.index[0]:
            continue
        
        # Skip rows that already have an EMA support alert (e.g., '13ema_recovery', '13ema_recover_failed', etc.)
        if pd.notna(df.loc[i, 'velocity_alert']):
            continue
        # Check for the 'velocity_maintained' alert (above both EMAs)
        if df.loc[i, 'above_13EMA'] and df.loc[i, 'above_169EMA']:
            df.loc[i, 'velocity_alert'] = 'velocity_maintained'

        # If price was below 13EMA in any of the past `window` periods and now it's above, set 'velocity_negotiating'
        elif (df.loc[i - window: i - 1, 'below_13EMA'].any() and df.loc[i, 'above_13EMA'])\
            or (df.loc[i - window: i - 1, 'below_169EMA'].any() and df.loc[i, 'above_169EMA']):
            df.loc[i, 'velocity_alert'] = 'velocity_negotiating'

        # Check for the 'velocity_loss' alert (below either 13EMA or 169EMA)
        elif df.loc[i, 'below_13EMA'] or df.loc[i, 'below_169EMA']:
            df.loc[i, 'velocity_alert'] = 'velocity_loss'

        # Check for the 'velocity_weak' alert (sandwiched between 13EMA and 169EMA)
        elif df.loc[i, 'Incremental_Low'] == 1:
            if df.loc[i - window: i - 1, 'above_13EMA'].any():
                df.loc[i, 'velocity_alert'] = 'velocity_weak'
        else:
            df.loc[i, 'velocity_alert'] = None
    # Drop intermediate columns used for conditions
    df.drop(columns=['above_13EMA', 'above_169EMA', 'below_13EMA', 'below_169EMA', 'between_13_169EMA'], inplace=True)

    return df
velocity_alert_df = velocity_alert(test_ema_support_alert_df)
# Get the rows with non-null alerts
non_null_alerts = velocity_alert_df.loc[velocity_alert_df['velocity_alert'].notnull()]

print(velocity_alert_df['velocity_alert'].value_counts())

print(velocity_alert_df['velocity_alert'].isnull().sum())

velocity_alert
velocity_loss            435
velocity_maintained      213
velocity_negotiating      80
169ema_recovery            7
13ema_recovery             6
13ema_recover_failed       6
169ema_recover_failed      6
Name: count, dtype: int64
3


In [17]:
velocity_alert_df

,date,169EMA,low,interval,13EMA,close,open,high,velocity_alert
0,2021-10-13,226.296006,268.593323,1,262.103969,270.359985,270.156677,271.803345,NaN
1,2021-10-14,226.842798,271.116669,1,263.628166,272.773346,271.829987,273.416656,NaN
2,2021-10-15,227.480059,274.116669,1,266.111286,281.010010,274.579987,281.070007,NaN
3,2021-10-18,228.216019,283.823334,1,269.529200,290.036682,283.929993,291.753326,velocity_maintained
4,2021-10-19,228.920419,287.503326,1,272.180742,288.089996,292.510010,292.649994,velocity_maintained
...,...,...,...,...,...,...,...,...,...
751,2024-10-09,214.646871,239.509995,1,245.294867,241.050003,243.820007,247.429993,velocity_loss
752,2024-10-10,214.930673,232.339996,1,244.362744,238.770004,241.809998,242.789993,velocity_loss
753,2024-10-11,214.964430,214.380005,1,240.568067,217.800003,220.130005,223.339996,13ema_recover_failed
754,2024-10-14,215.013790,213.740005,1,237.509772,219.160004,220.130005,221.910004,velocity_loss


### Anticipated Support driven by Velocity Movement

In [ ]:
# Fetch and prepare data for the selected stock
symbol = 'NVDA'
df = fetch_and_prepare_data(symbol)

# Compute the expected support alert
def anticipated_support(df):
    # If the price drops below the 13 EMA and cannot recover within 3 days
    for date in df.index:
        if df.loc[date,'close'] < df.loc[date, '13EMA'] and df.loc[date, 'recover_failed']:
            df.loc[date, 'anticipated_support'] = df.loc[date,'169EMA']

